# Text generation with Markovify

By [Allison Parrish](http://www.decontextualize.com/)

This notebook is a tour of how to generate text with Markov chains! Markov chains are a simple example of *predictive text generation*, a term I use to refer to methods of text generation that make use of statistical model that, given a certain stretch of text, *predicts* which bit of text should come next, based on probabilities learned from an existing corpus of text.

The code is written in Python, but you don't really need to know Python in order to use the notebook. Everything's pre-written for you, so you can just execute the cells, making small changes to the code as needed.

## Working with text files

Before we get started, we'll first need some text! Grab two [plain text files from Project Gutenberg](http://www.gutenberg.org/) (or from another source of your choice) and save them to the same directory as this notebook. (I suggest working with two files because we'll be running some code explicitly to "compare" two texts. Also, I think seeing two different outputs from the text generation methods discussed in this notebook will help you better understand how those methods work.) The code in the following cell loads into Python variables the contents of *two plain text files*, assigned to variables `text_a` and `text_b`. You'll need to replace the filenames with the names of the files that you downloaded, keeping the quotation marks (`"`) intact.

In [1]:
text_a = open("1342-0.txt").read()
text_b = open("84-0.txt").read()

These variables are *strings*, which are essentially just long lists of the characters that occur in the text, in the order that they occur. The code in the following cell shows the first two hundred characters of text A:

In [2]:
print(text_a[:200])

﻿The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away 


You can change `text_a` to `text_b` to see the output from your second text, or change `200` to a number of your choosing.

The `random.sample()` function gives us a random sampling of the contents of a variable (as long as that variable is a sequence of things, like a string or a list). So, for example, to see twenty random characters from text B:

In [3]:
import random
random.sample(text_b, 20)

['t',
 'd',
 'i',
 'h',
 ' ',
 's',
 's',
 'r',
 'e',
 'r',
 'i',
 'r',
 'd',
 's',
 'g',
 'v',
 'u',
 'd',
 'i',
 't']

This isn't incredibly helpful on its own, but you'll notice that the characters it drew (probably) more or less follow the expected letter distribution for English (i.e., lots of `e`s and `n`s and `t`s).

Perhaps more interesting would be to see a randomly-sampled list of *words*. To do this, we'll make separate variables for the words in the text, using a Python function called `.split()`, which takes a string and turns it into a list of words contained in that string. The following cell makes two new variables that contain the words from both texts respectively:

In [4]:
a_words = text_a.split()
b_words = text_b.split()

Now, ten random words from both text A and text B:

In [5]:
random.sample(a_words, 10)

['listened',
 'a',
 'see',
 'absence',
 'listening',
 'she',
 'as',
 'exhibit.”',
 'Fitzwilliam,',
 'state']

In [6]:
random.sample(b_words, 10)

['saw',
 'shepherd.',
 'only',
 'rustic,',
 'being',
 'resolved',
 'near',
 'would',
 'feel',
 'with']

The code in the following cell uses Python's `Counter` object to count the *most common* letters in the first of these texts:

In [7]:
from collections import Counter
Counter(text_a).most_common(12)

[(' ', 113941),
 ('e', 70344),
 ('t', 47283),
 ('a', 42156),
 ('o', 41138),
 ('n', 38430),
 ('i', 36273),
 ('h', 33883),
 ('r', 33293),
 ('s', 33292),
 ('d', 22247),
 ('l', 21282)]

Specifying the `a_words` variable gives the most frequent *words* instead:

In [8]:
Counter(a_words).most_common(12)

[('the', 4205),
 ('to', 4121),
 ('of', 3662),
 ('and', 3309),
 ('a', 1945),
 ('her', 1858),
 ('in', 1813),
 ('was', 1795),
 ('I', 1740),
 ('that', 1419),
 ('not', 1356),
 ('she', 1306)]

Compare these to the most common words in text B:

In [9]:
Counter(b_words).most_common(12)

[('the', 4056),
 ('and', 2971),
 ('of', 2741),
 ('I', 2719),
 ('to', 2142),
 ('my', 1631),
 ('a', 1394),
 ('in', 1125),
 ('was', 993),
 ('that', 987),
 ('with', 700),
 ('had', 679)]

## Markov models

I won't go into the precise details of how to implement a Markov chain text generator in this notebook. (I have written [a tutorial on this topic](https://github.com/aparrish/rwet/blob/master/ngrams-and-markov-chains.ipynb) elsewhere, however!) But I think it's helpful to understand the fundamentals of how Markov chain text generation works. The next question we’re going to try to answer is this: Given a stretch of text (say a string of characters, or run of words), what is most the most likely bit of text to come next?

One way to answer this question is with an n-gram based Markov model. What's an n-gram? I'm glad you asked!

### N-grams

An n-gram is simply a sequence of units drawn from a longer sequence; in the case of text, the unit in question is usually a character or a word. For convenience, we'll call the unit of the n-gram is called its level; the length of the n-gram is called its order. For example, the following is a list of all unique character-level order-2 n-grams in the word condescendences:

    co
    on
    nd
    de
    es
    sc
    ce
    en
    nc

And the following is an excerpt from the list of all unique word-level order-5 n-grams in The Road Not Taken:

    Two roads diverged in a
    roads diverged in a yellow
    diverged in a yellow wood,
    in a yellow wood, And
    a yellow wood, And sorry
    yellow wood, And sorry I

N-grams are used frequently in natural language processing and are a basic tool text analysis. Their applications range from programs that correct spelling to creative visualizations to compression algorithms to stylometrics to generative text.

### What comes next?

A Markov model for text begins with a list of n-grams. But in addition to making this list, we also keep track of what unit of text (word, character, etc.) *follows* each of those n-grams.

Let’s do a quick example by hand. This is the same character-level order-2 n-gram analysis of the (very brief) text “condescendences” as above, but this time keeping track of all characters that follow each n-gram:

| n-grams |	next? |
| ------- | ----- |
|co| n|
|on| d|
|nd| e, e|
|de| s, n|
|es| c, (end of text)|
|sc| e|
|ce| n, s|
|en| d, c|
|nc| e|

From this table, we can determine that while the n-gram `co` is followed by n 100% of the time, and while the n-gram `on` is followed by `d` 100% of the time, the n-gram `de` is followed by `s` 50% of the time, and `n` the rest of the time. Likewise, the n-gram `es` is followed by `c` 50% of the time, and followed by the end of the text the other 50% of the time.

Exercise: Imagine (or even better, write out) what this table might look like if you were analyzing words instead of characters, with a source text of your choice.

### Markov chains: Generating text from a Markov model

The Markov models we created above don't just give us interesting statistical probabilities. It also allows us generate a *new* text with those probabilities by *chaining together predictions*. Here’s how we’ll do it, starting with the order 2 character-level Markov model of `condescendences`: (1) start with the initial n-gram (`co`)—those are the first two characters of our output. (2) Now, look at the last *n* characters of output, where *n* is the order of the n-grams in our table, and find those characters in the “n-grams” column. (3) Choose randomly among the possibilities in the corresponding “next” column, and append that letter to the output. (Sometimes, as with `co`, there’s only one possibility). (4) If you chose “end of text,” then the algorithm is over. Otherwise, repeat the process starting with (2). Here’s a record of the algorithm in action:

    co
    con
    cond
    conde
    conden
    condend
    condendes
    condendesc
    condendesce
    condendesces
    
As you can see, we’ve come up with a word that looks like the original word, and could even be passed off as a genuine English word (if you squint at it). From a statistical standpoint, the output of our algorithm is nearly indistinguishable from the input. This kind of algorithm—moving from one state to the next, according to a list of probabilities—is known as a Markov chain generator.

### Generating with Markovify

Fortunately, with the invention of digital computers, you don't have to perform this algorithm by hand! In fact, Markov chain text generation has been a pastime of poets and programmers going back [all the way to 1983](https://www.jstor.org/stable/24969024), so it should be no surprise that there are many implementations of the idea in Python that you can download and install. The one we're going to use is [Markovify](https://github.com/jsvine/markovify), a Markov chain text generation library originally developed for BuzzFeed, apparently. It comes with a lot of extra niceties that will make our lives easier, but underneath the hood, it implements an algorithm very similar to the one we just did by hand above.

To install Markovify on your computer, run the cell below:

In [10]:
import sys
!{sys.executable} -m pip install markovify

And then run this cell to make the library available in your notebook:

In [11]:
import markovify

The code in the following cell creates a new text generator, using the text in the variable specified to build the Markov model, which is then assigned to the variable `generator_a`.

In [12]:
generator_a = markovify.Text(text_a)

You can then call the `.make_sentence()` method to generate a sentence from the model:

In [13]:
print(generator_a.make_sentence())

“I want to see his sister; and, oh! how ardently did she say?”


The `.make_short_sentence()` method allows you to specify a maximum length for the generated sentence:

In [14]:
print(generator_a.make_short_sentence(50))

As long as possible.


By default, Markovify tries to generate a sentence that is significantly different from any existing sentence in the input text. As a consequence, sometimes the `.make_sentence()` or `.make_short_sentence()` methods will return `None`, which means that in ten tries it wasn't able to generate such a sentence. You can work around this by increasing the number of times it tries to generate a sufficiently unique sentence using the `tries` parameter:

In [15]:
print(generator_a.make_short_sentence(40, tries=100))

Elizabeth was forced to rejoice.


Or by disabling the check altogether with `test_output=False` (note that this means the generator will occasionally return stretches of text that are present in the source text):

In [16]:
print(generator_a.make_short_sentence(40, test_output=False))

Mrs. Bennet, “that is rather singular.”


### Changing the order

When you create the model, you can specify the order of the model using the `state_size` parameter. It defaults to 2. Let's make two model with different orders and compare:

In [17]:
gen_a_1 = markovify.Text(text_a, state_size=1)
gen_a_4 = markovify.Text(text_a, state_size=4)

In [18]:
print("order 1")
print(gen_a_1.make_sentence(test_output=False))
print()
print("order 4")
print(gen_a_4.make_sentence(test_output=False))

order 1
“_My_ overhearings were miserable.

order 4
As they walked across the hall towards the river, Elizabeth turned back to look again; her uncle and aunt had already lost three days of happiness, and immediately wrote as follows: “I would have thanked you before, my dear aunt, as I ought to have done, for your long, kind, satisfactory, detail of particulars; but to say the truth, I was too cross to write.


In general, the higher the order, the more the sentences will seem "coherent" (i.e., more closely resembling the source text). Lower order models will produce more variation. Deciding on the order is usually a matter of taste and trial-and-error.

### Changing the level

Markovify, by default, works with *words* as the individual unit. It doesn't come out-of-the-box with support for character-level models. The following code defines a new kind of Markovify generator that implements character-level models. Execute it before continuing:

In [19]:
class SentencesByChar(markovify.Text):
    def word_split(self, sentence):
        return list(sentence)
    def word_join(self, words):
        return "".join(words)

Any of the parameters you passed to `markovify.Text` you can also pass to `SentencesByChar`. The `state_size` parameter still controls the order of the model, but now the n-grams are characters, not words.

The following cell implements a character-level Markov text generator for the word "condescendences":

In [20]:
con_model = SentencesByChar("condescendences", state_size=2)

Execute the cell below to see the output—it'll be a lot like what we implemented by hand earlier!

In [22]:
con_model.make_sentence()

'condescendescences'

Of course, you can use a character-level model on any text of your choice. So, for example, the following cell creates a character-level order-7 Markov chain text generator from text A:

In [23]:
gen_a_char = SentencesByChar(text_a, state_size=7)

And the cell below prints out a random sentence from this generator. (The `.replace()` is to get rid of any newline characters in the output.)

In [24]:
print(gen_a_char.make_sentence(test_output=False).replace("\n", " "))

As I must acknowledged, however, they can get.


### Combining models

Markovify has a handy feature that allows you to *combine* models, creating a new model that draws on probabilities from both of the source models. You can use this to create hybrid output that mixes the style and content of two (or more!) different source texts. To do this, you need to create the models independently, and then call `.combine()` to combine them.

In [25]:
generator_a = markovify.Text(text_a)
generator_b = markovify.Text(text_b)
combo = markovify.combine([generator_a, generator_b], [0.5, 0.5])

The bit of code `[0.5, 0.5]` controls the "weights" of the models, i.e., how much to emphasize the probabilities of any model. You can change this to suit your tastes. (E.g., if you want mostly text A with but a *soupçon* of text B, you would write `[0.9, 0.1]`. Try it!) 

Then you can create sentences using the combined model:

In [27]:
print(combo.make_sentence())

We all know that his religion and taught to look on the subject, but that the arguments with which I feared to meet her again.


### Bringing it all together

I've pre-written some code below to make it easy for you to experiment and produce output from Markovify. Just make adjustments to the values assigned to the variables in the cell below:

In [28]:
# change to "word" for a word-level model
level = "char"
# controls the length of the n-gram
order = 7
# controls the number of lines to output
output_n = 14
# weights between the models; text A first, text B second.
# if you want to completely exclude one model, set its corresponding value to 0
weights = [0.5, 0.5]
# limit sentence output to this number of characters
length_limit = 280

(The lines beginning with `#` are "comments"—they don't do anything, they're just there to explain what's happening in the code.)

After making your changes above, run the cell below to generate text according to your parameters. Repeat as necessary until you get something you really like!

In [29]:
model_cls = markovify.Text if level == "word" else SentencesByChar
gen_a = model_cls(text_a, state_size=order)
gen_b = model_cls(text_b, state_size=order)
gen_combo = markovify.combine([gen_a, gen_b], weights)
for i in range(output_n):
    out = gen_combo.make_short_sentence(length_limit, test_output=False)
    out = out.replace("\n", " ")
    print(out)
    print()

He was really instructed and repeatedly turned to Charlotte's first began.

My ancestors had manifested that would yet had been brought to provoke Darcy should think it a fainted.

Women fainted.

The completed; and its extreme profundity, until I almost as far enough to hold this explaining.

“So much mistaken in.

Your tempered, miserable elopement with extremest agitation for many years after breakfast a few stars, it will be in this likely to have them before the dead at my first shock of procure.

A military being perpetual fretting between the occasion requiring after _that_, I am a married, but a conversation with almost to London.

“I do not give us a ball.

In vain did Elizabeth ventured on me.”

In a few grey hairs covered them in the lane, who married.

Three years old.

“What a persuade him to the probably been soon to what promises fairly; and the church.

They soon cease to return Mr. Bingley's continued:  “Lizzy, let me be at home, therefore, to admire enough to endure; 

## Generating with non-prose text

Markovify assumes you're feeding it prose, i.e., a text file that can be parsed into sentences by separating on sentence-ending punctuation. But often you're *not* working with text like this. For example, let's generate some sonnets. First, download [this plaintext version of Shakespeare's sonnets](https://raw.githubusercontent.com/aparrish/plaintext-example-files/master/sonnets.txt) and keep it in the same directory as this notebook. We'll define the sonnet-generating task as consisting of (a) training a Markov chain on lines of poetry and then (b) generating a sequence of fourteen lines of poetry. Since the *line* is the unit now and not the *sentence*, we need to use Markovify's `NewlineText` class instead of `Text`:

In [41]:
sonnets_text = open("sonnets.txt").read()
sonnets_model = markovify.NewlineText(sonnets_text, state_size=1)

And then we can generate:

In [42]:
sonnets_model.make_sentence()

"For thy light's flame to make time's waste:"

And now make a sonnet, sorta:

In [44]:
for i in range(14):
    print(sonnets_model.make_sentence())

Thy pity like a face?
Or bends with all those.
Although thou art forced to thee, when he may not so of self-doing crime.
Could make you see aright?
For thee, as thou wilt look,
And therefore love, when first my verse alone kingdoms of self-doing crime.
Save breed, to say you a league is she might the gaudy spring,
The fairest creatures broke away,
Oaths of yours must strive
Therefore I fear,
Deserves the subject that thy years told:
Then happy me! but yet, like a seeting bath, which methinks I should example where I am to my love, that sorrow, which gives thee with winter hath taught it thy beauty herself is so long,
I better in thee to be forgot,
To give back again is took,


Doing this with a character-level model is a bit more tricky. I've written code in the cell below that defines a new class, `LinesByCharacter` that works like `NewlineText` but operates character-by-character instead of word-by-word:

In [45]:
class LinesByChar(markovify.NewlineText):
    def word_split(self, sentence):
        return list(sentence)
    def word_join(self, words):
        return "".join(words)

Now we can create a character model with the sonnets, line by line:

In [46]:
sonnets_char_model = LinesByChar(sonnets_text, state_size=4)

And generate new sonnets:

In [51]:
for i in range(14):
    print(sonnets_char_model.make_sentence())

One of hatred when in the be so blunt back nights in her proceeds:
And from thy fair, and see:
And faint of and the voice thy swear is best once with mask'd the deathese powerfumes invention, like a false with from her repair,
Which hall the night,
The leaven,
For take a hours, by title morn
The worth thee, form death desire is black lines thy deeds;
And all upon their riot ease child me not confound age's mine eye away,
My her.
Since canopy disgrace image infect.
Which now unfair witherein I do shall reness, where bore to temperfect thou away, year, when of a world's flatter stain,
It is pleast,
Lo! in their breat true my mother plead where fresh, who art compounds,
Doubting lack of bloody tyrannot action but drown vision laughts, are of your critied be.


### New moods

Character-level Markov chains are especially suitable, in my experience, for generating shorter texts, like individual words or names. Let's generate names of new moods using this technique. First, download [this JSON file of moods](https://raw.githubusercontent.com/dariusk/corpora/9bb62927951f79bec2454f29d71b6e9b28d874b1/data/humans/moods.json) from [Corpora Project](https://github.com/dariusk/corpora/) and save to the same directory as this notebook.

Then load the JSON file and grab just the list of words naming moods:

In [52]:
import json
mood_data = json.loads(open("./moods.json").read())
moods = mood_data['moods']

The easiest way to use this is to make one big string with the moods joined together with newlines:

In [53]:
moods_text = "\n".join(moods)

Then use `LinesByChar` to make the model:

In [56]:
moods_char_model = LinesByChar(moods_text, state_size=3)

And voila, new moods:

In [57]:
for i in range(24):
    print(moods_char_model.make_sentence())

cowarlessive
toughted
lyricapative
convictorted
embarratived
teasane
feistative
powerloaded
bothetic
rejuvenaccepted
teassioned
innovatired
chieved
alievoless
incomperational
greterpressived
imped
soreborn
lethant
jadequashamed
intemplifeless
deterrientalkatirical
enconfidead
strappallous


## Further reading

* Hayes, Brian. “Computer recreations.” Scientific American, vol. 249, no. 5, 1983, pp. 18–31. JSTOR, http://www.jstor.org/stable/24969024. (Original column from Scientific American that described how Markov chain text generation works—very readable! I can send a PDF, hit me up.)
* [A Travesty Generator for Micros](https://elmcip.net/critical-writing/travesty-generator-micros) is a follow-up to Hayes' article that has some more theory and an actual Pascal listing (which is now mostly of only historical interest).
* [This notebook](https://github.com/aparrish/rwet/blob/master/ngrams-and-markov-chains.ipynb) shows how to implement a Markov chain generator from scratch in Python, if you're interested in such things!
* Lillian-Yvonne Bertram's [Travesty Generator](http://www.noemipress.org/catalog/poetry/travesty-generator/) is a striking example of Markov chains put to poetic use.